# Structured Streaming using the Python DataFrames API

Apache Spark includes a high-level stream processing API, [Structured Streaming](http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html). In this notebook we take a quick look at how to use the DataFrame API to build Structured Streaming applications. We want to compute real-time metrics like running counts and windowed counts on a stream of timestamped actions (e.g. Open, Close, etc).

To run this notebook, import it and attach it to a Spark cluster.

## Sample Data
We have some sample action data as files in `/databricks-datasets/structured-streaming/events/` which we are going to use to build this appication. Let's take a look at the contents of this directory.

In [0]:
# Look at the content of the following folder: /databricks-datasets/structured-streaming/events/
# using the following command: display(dbutils.fs.ls('dbfs:/databricks-datasets/structured-streaming/events/')
# What do you see?
display(dbutils.fs.ls("/databricks-datasets/structured-streaming/events/"))


path,name,size,modificationTime
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008,1596690605000


There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [0]:
# Look at the functions head in dbutils
dbutils.help()
# Open one file
display(spark.read.json("/databricks-datasets/structured-streaming/events/file-0.json"))

This module provides various utilities for users to interact with the rest of Databricks.
 credentials: DatabricksCredentialUtils -> Utilities for interacting with credentials within notebooks data: DataUtils -> Utilities for understanding and interacting with datasets (EXPERIMENTAL) fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console jobs: JobsUtils -> Utilities for leveraging jobs features library: LibraryUtils -> Utilities for session isolated libraries meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

action,time
Open,1469501107
Open,1469501147
Open,1469501202
Open,1469501219
Open,1469501225
Open,1469501234
Open,1469501245
Open,1469501246
Open,1469501248
Open,1469501256


Each line in the file contains JSON record with two fields - `time` and `action`. Let's try to analyze these files interactively.

## Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

In [0]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType(
  [ StructField("time", TimestampType(), True),
   StructField("action", StringType(), True) ]
)



In [0]:
# Read all json files, taking into account the defined schema, and display the content 
staticInputDF = spark.read.schema(jsonSchema).json(inputPath)
display(staticInputDF)


time,action
2016-07-28T04:19:28.000Z,Close
2016-07-28T04:19:28.000Z,Close
2016-07-28T04:19:29.000Z,Open
2016-07-28T04:19:31.000Z,Close
2016-07-28T04:19:31.000Z,Open
2016-07-28T04:19:31.000Z,Open
2016-07-28T04:19:32.000Z,Close
2016-07-28T04:19:33.000Z,Close
2016-07-28T04:19:35.000Z,Close
2016-07-28T04:19:36.000Z,Open


- Compare the dates from the output without schema and with it. 
- Did you notice that inputPath is a folder?

In [0]:
# Calculate the total number of 'Open' and 'Close' actions 
from pyspark.sql.functions import col

staticInputDF.groupBy("action").count().show()

+------+-----+
|action|count|
+------+-----+
| Close|50000|
|  Open|50000|
+------+-----+



In [0]:
# Determine min and max time
from pyspark.sql.functions import min, max

staticInputDF.select(min("time").alias("min_time"), max("time").alias("max_time")).show()

+-------------------+-------------------+
|           min_time|           max_time|
+-------------------+-------------------+
|2016-07-26 02:45:07|2016-07-28 06:48:19|
+-------------------+-------------------+



In [0]:
# Calculate the number of "open" and "close" actions with one hour windows: staticCountsDF
# Look at groupBy(..., window) function

from pyspark.sql.functions import window

staticCountsDF = (
    staticInputDF
    .groupBy(window("time", "1 hour"), "action")
    .count()
)

display(staticCountsDF)

window,action,count
"List(2016-07-27T03:00:00.000Z, 2016-07-27T04:00:00.000Z)",Open,1014
"List(2016-07-27T03:00:00.000Z, 2016-07-27T04:00:00.000Z)",Close,1025
"List(2016-07-28T04:00:00.000Z, 2016-07-28T05:00:00.000Z)",Close,960
"List(2016-07-27T23:00:00.000Z, 2016-07-28T00:00:00.000Z)",Open,1008
"List(2016-07-27T23:00:00.000Z, 2016-07-28T00:00:00.000Z)",Close,1011
"List(2016-07-26T13:00:00.000Z, 2016-07-26T14:00:00.000Z)",Close,1028
"List(2016-07-26T11:00:00.000Z, 2016-07-26T12:00:00.000Z)",Open,991
"List(2016-07-27T12:00:00.000Z, 2016-07-27T13:00:00.000Z)",Close,1035
"List(2016-07-26T14:00:00.000Z, 2016-07-26T15:00:00.000Z)",Open,991
"List(2016-07-27T20:00:00.000Z, 2016-07-27T21:00:00.000Z)",Open,1005


In [0]:
# Make this window a sliding window (30 minutes overlap): staticCountsSW
staticCountsSW = (
    staticInputDF
    .groupBy(window("time", "1 hour", "30 minutes"), "action")
    .count()
)

display(staticCountsSW)

window,action,count
"List(2016-07-26T13:30:00.000Z, 2016-07-26T14:30:00.000Z)",Close,1042
"List(2016-07-27T20:30:00.000Z, 2016-07-27T21:30:00.000Z)",Close,1001
"List(2016-07-27T03:30:00.000Z, 2016-07-27T04:30:00.000Z)",Close,1079
"List(2016-07-27T03:00:00.000Z, 2016-07-27T04:00:00.000Z)",Open,1014
"List(2016-07-27T03:00:00.000Z, 2016-07-27T04:00:00.000Z)",Close,1025
"List(2016-07-28T04:00:00.000Z, 2016-07-28T05:00:00.000Z)",Close,960
"List(2016-07-27T23:00:00.000Z, 2016-07-28T00:00:00.000Z)",Open,1008
"List(2016-07-27T23:00:00.000Z, 2016-07-28T00:00:00.000Z)",Close,1011
"List(2016-07-28T06:30:00.000Z, 2016-07-28T07:30:00.000Z)",Close,33
"List(2016-07-26T13:00:00.000Z, 2016-07-26T14:00:00.000Z)",Close,1028


In [0]:
# Register staticCountsDF (createOrReplaceTempView) as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")


Now we can directly use SQL to query the table.

In [0]:
%sql
-- Count all Open and Close actions in the table static_counts  
SELECT action, SUM(count) AS total_count
FROM static_counts
GROUP BY action
ORDER BY action;

action,total_count
Close,50000
Open,50000


In [0]:
%sql
-- How many actions (Close and Open separately) is within each time window (in the table static_counts)
SELECT 
  window.start AS window_start,
  window.end AS window_end,
  action,
  count
FROM static_counts
ORDER BY window.start, action;
-- Make a plot


window_start,window_end,action,count
2016-07-26T02:00:00.000Z,2016-07-26T03:00:00.000Z,Close,11
2016-07-26T02:00:00.000Z,2016-07-26T03:00:00.000Z,Open,179
2016-07-26T03:00:00.000Z,2016-07-26T04:00:00.000Z,Close,344
2016-07-26T03:00:00.000Z,2016-07-26T04:00:00.000Z,Open,1001
2016-07-26T04:00:00.000Z,2016-07-26T05:00:00.000Z,Close,815
2016-07-26T04:00:00.000Z,2016-07-26T05:00:00.000Z,Open,999
2016-07-26T05:00:00.000Z,2016-07-26T06:00:00.000Z,Close,1003
2016-07-26T05:00:00.000Z,2016-07-26T06:00:00.000Z,Open,1000
2016-07-26T06:00:00.000Z,2016-07-26T07:00:00.000Z,Close,1011
2016-07-26T06:00:00.000Z,2016-07-26T07:00:00.000Z,Open,993


Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

**NOTE:** Due to the Databricks environment, it is needed to also create temp tables in the unity catalog by running the below cell before starting the streaming processing. Additionally the below cell will need to be re-run to clear out the temp tables prior to re-running any  streaming dfs.

In [0]:
# Define the name of the new catalog
catalog = 'workspace'

# define variables for the trips data
schema = 'default'
volume = 'checkpoints'

# Path for file operations
path_volume = f'/Volumes/{catalog}/{schema}/{volume}'

# Three-part names for SQL operations
path_table = f'{catalog}.{schema}'
volume_name = f'{catalog}.{schema}.{volume}'

# Drop old temp volume (use three-part name, not path)
spark.sql(f"DROP VOLUME IF EXISTS {volume_name}")

# Create new temp volume
spark.sql(f"CREATE VOLUME IF NOT EXISTS {volume_name}")

# Define tmp dir for each stream
tmp_input = f"{path_volume}/input"
tmp_streaming_counts = f"{path_volume}/streaming_counts"
tmp_streaming_counts_filter = f"{path_volume}/streaming_counts_filter"
tmp_streaming_counts_run = f"{path_volume}/streaming_counts_run"

## Demo: Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.


In [0]:
from pyspark.sql.functions import *

# Read data from a file
# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Do some transformations
# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

True

In [0]:
# Display input data
streamingInputDF.display(checkpointLocation=tmp_input)

time,action
2016-07-26T02:45:07.000Z,Open
2016-07-26T02:45:47.000Z,Open
2016-07-26T02:46:42.000Z,Open
2016-07-26T02:46:59.000Z,Open
2016-07-26T02:47:05.000Z,Open
2016-07-26T02:47:14.000Z,Open
2016-07-26T02:47:25.000Z,Open
2016-07-26T02:47:26.000Z,Open
2016-07-26T02:47:28.000Z,Open
2016-07-26T02:47:36.000Z,Open


In [0]:
# Display transformed data
streamingCountsDF.display(checkpointLocation=tmp_streaming_counts)

action,window,count
Close,"List(2016-07-26T13:00:00.000Z, 2016-07-26T14:00:00.000Z)",1028
Open,"List(2016-07-26T18:00:00.000Z, 2016-07-26T19:00:00.000Z)",1004
Close,"List(2016-07-27T02:00:00.000Z, 2016-07-27T03:00:00.000Z)",971
Open,"List(2016-07-27T04:00:00.000Z, 2016-07-27T05:00:00.000Z)",995
Open,"List(2016-07-27T05:00:00.000Z, 2016-07-27T06:00:00.000Z)",986
Open,"List(2016-07-26T05:00:00.000Z, 2016-07-26T06:00:00.000Z)",1000
Open,"List(2016-07-26T11:00:00.000Z, 2016-07-26T12:00:00.000Z)",991
Close,"List(2016-07-26T06:00:00.000Z, 2016-07-26T07:00:00.000Z)",1011
Close,"List(2016-07-27T05:00:00.000Z, 2016-07-27T06:00:00.000Z)",987
Open,"List(2016-07-26T10:00:00.000Z, 2016-07-26T11:00:00.000Z)",1007


In [0]:
# Add aditional filter to transformed dataframe
streamingCountsDF.filter(streamingCountsDF.action == 'Open').display(checkpointLocation=tmp_streaming_counts_filter)

action,window,count
Open,"List(2016-07-26T18:00:00.000Z, 2016-07-26T19:00:00.000Z)",1004
Open,"List(2016-07-27T04:00:00.000Z, 2016-07-27T05:00:00.000Z)",995
Open,"List(2016-07-27T05:00:00.000Z, 2016-07-27T06:00:00.000Z)",986
Open,"List(2016-07-26T05:00:00.000Z, 2016-07-26T06:00:00.000Z)",1000
Open,"List(2016-07-26T11:00:00.000Z, 2016-07-26T12:00:00.000Z)",306
Open,"List(2016-07-26T10:00:00.000Z, 2016-07-26T11:00:00.000Z)",1007
Open,"List(2016-07-27T07:00:00.000Z, 2016-07-27T08:00:00.000Z)",998
Open,"List(2016-07-27T18:00:00.000Z, 2016-07-27T19:00:00.000Z)",995
Open,"List(2016-07-27T19:00:00.000Z, 2016-07-27T20:00:00.000Z)",1007
Open,"List(2016-07-28T03:00:00.000Z, 2016-07-28T04:00:00.000Z)",996


As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table 
    .queryName("counts")     # counts = name of the in-memory table
    .option("checkpointLocation", tmp_streaming_counts_run)
    .outputMode("complete")  # complete = all the counts should be in the table
    .trigger(availableNow=True)
    .start()
)

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. The progress bar shows that the query is active. 
Furthermore, if you expand the `> counts` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [0]:
%sql
SELECT *
FROM counts

action,window,count


In [0]:
from time import sleep
sleep(1)  # wait a bit for computation to start

In [0]:
%sql
select action, date_format(window.end, "MMM-dd HH:mm") as time, count
from counts
order by time, action

action,time,count


We see the timeline of windowed counts (similar to the static one earlier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [0]:
sleep(1)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,176
Open,Jul-26 05:00,289


In [0]:
sleep(1)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,323
Open,Jul-26 06:00,328
Close,Jul-26 14:00,699
Open,Jul-26 14:00,656


Also, let's see the total number of "opens" and "closes".

In [0]:
%sql 
select action, sum(count) as total_count 
from counts 
group by action 
order by action

action,total_count
Close,5514
Open,6486


If you keep running the above query repeatedly, you will always find that the number of "opens" is more than the number of "closes", as expected in a data stream where a "close" always appear after corresponding "open". This shows that Structured Streaming ensures **prefix integrity**. Read the blog posts linked below if you want to know more.

Note that there are only a few files, so consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.

Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing `query.stop()`. Either way, when the query is stopped, the status of the corresponding cell above will automatically update to `TERMINATED`.

In [0]:
query.stop()

### IoT data

Develop a streaming example on `IoT device`dataset:

- inspect the dataset
- ask yourself couple of questions about the data and try to answer them (eg. how many steps users do, how many calories do they burn...)
- you read the data in streaming fashion (file by file) and keep the data for only one company? Here are some hints:
  - you can find the schema in the readme file 
  - as above, use this option: .option("maxFilesPerTrigger", 1)
  - use user_id or device_id for grouping
  - use timestamp for window definition
  - you can try streaming joins with the user data (/databricks-datasets/iot-stream/data-user/userData.csv). Here is the doc: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#join-operations

In [0]:
# Iot stream dataset
display(dbutils.fs.ls('/databricks-datasets/iot-stream/'))

path,name,size,modificationTime
dbfs:/databricks-datasets/iot-stream/README.md,README.md,1596,1596565744000
dbfs:/databricks-datasets/iot-stream/data-device/,data-device/,0,1762534514718
dbfs:/databricks-datasets/iot-stream/data-user/,data-user/,0,1762534514718


In [0]:
# Read the README file

In [0]:
# Define the schema (copy from the README) data-device

In [0]:
# Open one file to see how the data looks like (as a static dataframe)

In [0]:
# Define your streaming dataframe

In [0]:
# Define your transformations

In [0]:
# Define the sink and start streaming 

In [0]:
%sql
-- visualize your streaming analytics